In [1]:
# libraries from finetuning_parameters.py
from finetuning_parameters import get_args
from future.baseline_trainer import BaselineTuner
from future.modules import ptl2classes
from future.hooks import EvaluationRecorder

from data_loader.wrap_sampler import wrap_sampler
import data_loader.task_configs as task_configs
import data_loader.data_configs as data_configs
from future.collocate_fns import task2collocate_fn

import utils.checkpoint as checkpoint
import utils.logging as logging

import torch
import random
import os

# libraries from future/base.py
from torch.utils.data import SequentialSampler, RandomSampler
from future.hooks import EvaluationRecorder
import utils.eval_meters as eval_meters
from seqeval.metrics import f1_score as f1_score_tagging
import torch

# libraries from future/baseline_trainer.py
import torch
import torch.nn as nn
import numpy as np
from copy import deepcopy
from future.base import BaseTrainer
from future.hooks.base_hook import HookContainer
from future.hooks import EvaluationRecorder
from torch.utils.data import RandomSampler
from collections import defaultdict, Counter
from tqdm import tqdm

# and so on..
from finetuning_baseline import init_config, init_task, init_hooks

2022-03-02 01:27:23.545861: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/ssl/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nvidia/lib64:/usr/include/x86_64-linux-gnu
2022-03-02 01:27:23.545886: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
parser = get_args()
conf = parser.parse_args(args=[])

In [20]:
conf.dataset_name = 'xnli'
conf.trn_languages = 'english'
if conf.dataset_name == 'pawsx':
    conf.eval_languages = 'english,german,chinese,french,japanese,korean,spanish'
elif conf.dataset_name == 'xnli':
    conf.eval_languages = 'english,arabic,bulgarian,chinese,french,german,greek,hindi,russian,spanish,swahili,thai,turkish,urdu,vietnamese'
conf.finetune_epochs = 10
conf.finetune_batch_size = 256
conf.eval_every_batch = 50
conf.override = False
conf.train_fast = False
conf.world = '0'
conf.finetune_lr = 1e-5

In [21]:
init_config(conf)
model, tokenizer, data_iter, metric_name, collocate_batch_fn = init_task(conf)

data_iter

./checkpoint_baseline/xnli/debug/1646186597_model_task-xnli_flr-1.0E-05_ftbs-256_ftepcs-10_sd-3_trnfast-False_evalevery-50_tlang-en_vlang-en-ar-bg-zh-fr-de-el-hi-ru-es-sw-th-tr-ur-vi


parameters: 
	override	False
	experiment	debug
	ptl	bert
	model	bert-base-uncased
	dataset_name	xnli
	max_seq_len	128
	trn_languages	['english']
	eval_languages	['english', 'arabic', 'bulgarian', 'chinese', 'french', 'german', 'greek', 'hindi', 'russian', 'spanish', 'swahili', 'thai', 'turkish', 'urdu', 'vietnamese']
	finetune_epochs	10
	eval_every_batch	50
	finetune_batch_size	256
	finetune_lr	1e-05
	inference_batch_size	512
	data_path	./data/download
	checkpoint	./checkpoint_baseline
	manual_seed	3
	summary_freq	100
	time_stamp	None
	train_fast	False
	track_time	True
	world	[0]
	is_finished	False
	task	xnli
	n_sub_process	1
	time_stamp_	1646186597_model_task-xnli_flr-1.0E-05_ftbs-256_ftepcs-10_sd-3_trnfast-False_evalevery-50_tlang-en_vlang-en-ar-bg-zh-fr-de-el-hi-ru-es-sw-th-tr-ur-vi
	checkpoint_root	.

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.


{'arabic': <data_loader.data_iters.SeqClsDataIter at 0x7f38559356d0>,
 'bulgarian': <data_loader.data_iters.SeqClsDataIter at 0x7f380fb17f10>,
 'chinese': <data_loader.data_iters.SeqClsDataIter at 0x7f380fa87550>,
 'english': <data_loader.data_iters.SeqClsDataIter at 0x7f380f31b390>,
 'french': <data_loader.data_iters.SeqClsDataIter at 0x7f380ef03c10>,
 'german': <data_loader.data_iters.SeqClsDataIter at 0x7f382315d7d0>,
 'greek': <data_loader.data_iters.SeqClsDataIter at 0x7f3822e30890>,
 'hindi': <data_loader.data_iters.SeqClsDataIter at 0x7f380c256290>,
 'russian': <data_loader.data_iters.SeqClsDataIter at 0x7f3823262750>,
 'spanish': <data_loader.data_iters.SeqClsDataIter at 0x7f3841a0ca50>,
 'swahili': <data_loader.data_iters.SeqClsDataIter at 0x7f38235ebe10>,
 'thai': <data_loader.data_iters.SeqClsDataIter at 0x7f380ef4a710>,
 'turkish': <data_loader.data_iters.SeqClsDataIter at 0x7f380ef29310>,
 'urdu': <data_loader.data_iters.SeqClsDataIter at 0x7f380c612310>,
 'vietnamese': <d

In [22]:
adapt_loaders = {}
for language, language_dataset in data_iter.items():
    # NOTE: the sample dataset are refered
    adapt_loaders[language] = wrap_sampler(
        trn_batch_size=conf.finetune_batch_size,
        infer_batch_size=conf.inference_batch_size,
        language=language,
        language_dataset=language_dataset,
    )
hooks = init_hooks(conf, metric_name)

[WARN] trn_egs of arabic has zero egs
[WARN] trn_egs of bulgarian has zero egs
[WARN] trn_egs of chinese has zero egs
[WARN] trn_egs of french has zero egs
[WARN] trn_egs of german has zero egs
[WARN] trn_egs of greek has zero egs
[WARN] trn_egs of hindi has zero egs
[WARN] trn_egs of russian has zero egs
[WARN] trn_egs of spanish has zero egs
[WARN] trn_egs of swahili has zero egs
[WARN] trn_egs of thai has zero egs
[WARN] trn_egs of turkish has zero egs
[WARN] trn_egs of urdu has zero egs
[WARN] trn_egs of vietnamese has zero egs


In [23]:
trainer = BaselineTuner(
        conf, collocate_batch_fn=collocate_batch_fn, logger=conf.logger
    )
trainer.conf.eval_languages

2022-03-02 02:03:43	Init trainer.


['english',
 'arabic',
 'bulgarian',
 'chinese',
 'french',
 'german',
 'greek',
 'hindi',
 'russian',
 'spanish',
 'swahili',
 'thai',
 'turkish',
 'urdu',
 'vietnamese']

In [24]:
if conf.dataset_name == 'xnli':
    PATH='/home/vessl/FSXLT/_checkpoint_baseline/xnli/debug/1645106190_model_task-xnli_flr-3.0E-05_ftbs-32_ftepcs-10_sd-3_trnfast-False_evalevery-50_tlang-en_vlang-en/state_dicts/best_state.pt'
elif conf.dataset_name == 'pawsx':
    PATH='/home/vessl/FSXLT/_checkpoint_baseline/pawsx/debug/1645085231_model_task-pawsx_flr-1.0E-05_ftbs-32_ftepcs-10_sd-3_trnfast-False_evalevery-50_tlang-en_vlang-en/state_dicts/best_state.pt'
model.load_state_dict(torch.load(PATH)['best_state_dict'], strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['bert.embeddings.position_ids'])

In [25]:
# trainer.train
opt, model = trainer._init_model_opt(model)
trainer.model = model
trainer.model.eval()

# for epoch_index in tqdm(range(1, 1 + 1)):
    
for language in trainer.conf.eval_languages: # trn_languages
    print ("Start language-{}".format(language))
    labels = np.empty((0,))
    features = np.empty((0, 768))
    trn_iters = []
    egs = adapt_loaders[language].val_egs # trn_egs
    trn_iters.append(iter(egs))
    
    correct = 0.
    total = 0.

    batches_per_epoch = max(len(ti) for ti in trn_iters)
    for batch_index in range(1, batches_per_epoch + 1):
        trn_loss = []
        for ti in trn_iters:
            try:
                batched = next(ti)
            except StopIteration:
                continue
            batched, golds, uids, _golds_tagging = trainer.collocate_batch_fn(
                batched
            )
            with torch.no_grad():
                logits = trainer.model(**batched)
                correct += (logits[0].max(1)[1] == golds).sum()
                total += len(logits[0])
                
    print (correct / total * 100)

Initialize Optimizer and Model
model = model.cuda()
Start language-english
tensor(85.7831, device='cuda:0')
Start language-arabic
tensor(33.8153, device='cuda:0')
Start language-bulgarian
tensor(34.6586, device='cuda:0')
Start language-chinese
tensor(34.6185, device='cuda:0')
Start language-french
tensor(40.9639, device='cuda:0')
Start language-german
tensor(37.6305, device='cuda:0')
Start language-greek
tensor(34.8193, device='cuda:0')
Start language-hindi
tensor(33.8554, device='cuda:0')
Start language-russian
tensor(34.3775, device='cuda:0')
Start language-spanish
tensor(43.0924, device='cuda:0')
Start language-swahili
tensor(38.6747, device='cuda:0')
Start language-thai
tensor(35.6627, device='cuda:0')
Start language-turkish
tensor(38.3534, device='cuda:0')
Start language-urdu
tensor(33.6546, device='cuda:0')
Start language-vietnamese
tensor(37.1888, device='cuda:0')
